## Inferencing Script

This script is used in combination with a trained model (here, `retrained_graph.pb`) and its labels (`retrained_labels.txt`) to classify a new image input.

Utilization from the command line: 

`>> python path/to/labeler.py /path/to/test_image.jpg`

In [ ]:
import tensorflow as tf, sys

# Takes in command line input.
image_path = sys.argv[1]

# Read in the image data.
image_data = tf.gfile.FastGFile(image_path, 'rb').read()

# Loads the label file and strips off the carriage returns.
label_lines = [line.rstrip() for line in tf.gfile.GFile("/insert/your/path/to/retrained_labels.txt")]

# Unpersists the graph from file.
with tf.gfile.FastGFile("/insert/your/path/to/retrained_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')
    
# Feeds the image data as input to the graph, and gets the first prediction.
with tf.Session() as sess:
    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
    predictions = sess.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})
    
    # Sort to show labels of first prediction in order of confidence
    top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]

    for node_id in top_k:
        human_string = label_lines[node_id]
        score = predictions[0][node_id]
        print('%s (score = %.5f)' % (human_string, score))